# NLP con Hugging Face

## Procesando los datos para NLP

### Descargando el dataset

Usaremos el dataset MRPC. Este es uno de los 10 datasets que componen el [benchmark (punto de referencia) GLUE](https://huggingface.co/datasets/glue). Se utiliza para medir el rendimiento de los modelos ML en 10 tareas de clasificación de texto diferentes.

En otras palabras, seleccionamos el subset `mrpc` del dataset `glue`:

In [1]:
from datasets import load_dataset

ds = load_dataset("glue", "mrpc")

Así se ve un ejemplo. Notamos que `mrpc` está compuesto de dos oraciones y una etiqueta que indica si los dos enunciados son equivalentes.

In [2]:
ex = ds["train"][400]
ex

{'sentence1': 'U.S. Agriculture Secretary Ann Veneman , who announced Tuesdays ban , also said Washington would send a technical team to Canada to help .',
 'sentence2': "U.S. Agriculture Secretary Ann Veneman , who announced yesterday 's ban , also said Washington would send a technical team to Canada to assist in the Canadian situation .",
 'label': 1,
 'idx': 446}

In [3]:
labels = ds["train"].features["label"]
labels

ClassLabel(names=['not_equivalent', 'equivalent'], id=None)

In [4]:
labels.int2str(1)

'equivalent'

### Tokenizando

¿Recuerdas que con visión descargamos el feature extractor directamente del repositorio del modelo pre-entrenado que vamos a usar como base?

Podemos pensar en la función tokenizadora como el equivalente en el NLP.

Descargamos el tokenizador directamente del repo del modelo que usaremos.

In [5]:
from transformers import AutoTokenizer

# Modelo
repo_id = "bert-base-uncased"

# Tokenizador
tokenizer = AutoTokenizer.from_pretrained(repo_id)

Para preprocesar el conjunto de datos necesitamos convertir el texto en números que el modelo pueda entender. Esto se hace con un tokenizador.

Pasar de texto a números se conoce como codificación o encoding. El encoding se realiza en un proceso de dos pasos: la tokenización, seguida de la conversión a input ids. Por el momento nos basta saber que estamos traduciendo texto a números llamados como input ids. Estos estarán en el formato adecuado para alimentar nuestro modelo.

Podemos alimentar al tokenizador con una oración o una lista de oraciones, por lo que podemos tokenizar directamente todas las primeras oraciones y todas las segundas oraciones de cada par de esta manera:

In [6]:
tokenized_sentence_1 = tokenizer(ds["train"]["sentence1"][2])
tokenized_sentence_1

{'input_ids': [101, 2027, 2018, 2405, 2019, 15147, 2006, 1996, 4274, 2006, 2238, 2184, 1010, 5378, 1996, 6636, 2005, 5096, 1010, 2002, 2794, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Necesitamos manejar los dos enunciados como un par y no separados. El tokenizador puede tomar un par de secuencias y prepararlas de la manera que espera nuestro modelo:

In [7]:
# tokenizamos dos frases
inputs = tokenizer("This is a sample sentence", "Another sample sentence")
inputs

{'input_ids': [101, 2023, 2003, 1037, 7099, 6251, 102, 2178, 7099, 6251, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

¿Qué significa cada uno de los valores que nos retorna el tokenizador?
- `input_ids` es la traducción de palabras a números.
- `attention_mask` es un tensor con la misma forma que `input_ids`, pero lleno de 0 y 1: los 1 indican que se debe atender a los tokens correspondientes y los 0 indican que no se deben atender. Es decir, deben ser ignorados por el modelo.
- `token_type_ids` dice al modelo qué parte de la entrada es la primera oración y cuál es la segunda oración.

El modelo espera que las entradas sean de la forma [CLS] oración 1 [SEP] oración 2 [SEP] cuando hay dos oraciones.

In [8]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'a',
 'sample',
 'sentence',
 '[SEP]',
 'another',
 'sample',
 'sentence',
 '[SEP]']

Si seleccionamos otro modelo en el Hub no necesariamente tendremos `token_type_ids` en las entradas tokenizadas (por ejemplo, no se devuelven si usa un modelo `DistilBERT`). Solo se devuelven cuando el modelo sabrá qué hacer con ellas, porque los ha visto durante su preentrenamiento.

En general, no necesitamos preocuparnos por si hay o no `token_type_ids` en nuestras entradas tokenizadas, siempre que usemos el tokenizador correspondiente al modelo, todo estará bien ya que el tokenizador sabe qué proporcionar al modelo.

Por ejemplo, durante esta clase utilizaremos un modelo [`distilroberta-base`](https://huggingface.co/distilroberta-base) por su tamaño y efectividad. Pero no cuenta con `token_type_ids` y aún así nos regresa excelentes resultados.

En la organización del Platzi en el Hub puedes encontrar un [modelo BERT](https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel) afinado siguiendo el mismo proceso que usamos en esta clase.

In [9]:
# Modelo
repo_id = "distilroberta-base"

# Tokenizador
tokenizer = AutoTokenizer.from_pretrained(repo_id)

Creamos una función tokenizadora. Recibe un ejemplo y lo tokeniza.

In [10]:
def tokenize_fn(examples):
    return tokenizer(
        examples["sentence1"], examples["sentence2"], # frases
        truncation=True # truncamos las frases al máximo permitido
    )

In [11]:
# tokenizamos el dataset
prepared_ds = ds.map(tokenize_fn, batched=True)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

### Definiendo el data collator: Dynamic padding

Necesitamos que nuestros tensores tengan una forma rectangular. Es decir que tengan el mismo tamaño cada uno de los ejemplos. Sin embargo, los textos no necesariamente tienen el mismo tamaño.

Para ello usamos el relleno o padding. El padding se asegura de que todas nuestras oraciones tengan la misma longitud al agregar una palabra especial llamada padding token a las oraciones con menos valores. Por ejemplo, si tenemos 10 oraciones con 10 palabras y 1 oración con 20 palabras, el relleno garantizará que todas las oraciones tengan 20 palabras.

Dejamos el argumento de `padding` del tokenizer vacío en nuestra función de tokenización por ahora. Esto se debe a que rellenar (hacer padding) todas las muestras hasta la longitud máxima del dataset no es eficiente, es mejor rellenar las muestras cuando estamos construyendo un batch, ya que entonces solo necesitamos rellenar hasta la longitud máxima en ese batch, y no la longitud máxima en todo el dataset. ¡Esto puede ahorrar mucho tiempo y potencia de procesamiento cuando las entradas tienen longitudes muy variables!

Usaremos un DataCollator para esto.

Rellenemos (hagamos padding) todos los ejemplos con la longitud del elemento más largo del batch. A esta técnica se le conoce como relleno dinámico o dynamic padding.

In [12]:
from transformers import DataCollatorWithPadding

# creamos el collator
data_collator = DataCollatorWithPadding(tokenizer)

## Entrenamiento y evaluación

Definamos el resto de los argumentos necesarios para `Trainer`.

### Definiendo la métrica

In [13]:
import evaluate
import numpy as np

# Metrica
def compute_metrics(eval_pred):
    metric = evaluate.load("glue", "mrpc") # cargamos la metrica
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # obtenemos las predicciones
    return metric.compute(predictions=predictions, references=labels) # calculamos la metrica

### Configurando `Trainer`


In [14]:
from transformers import AutoModelForSequenceClassification

# Etiquetas
labels = ds["train"].features["label"].names

# Cargamos el modelo
model = AutoModelForSequenceClassification.from_pretrained(
    repo_id, # id del modelo
    num_labels = len(labels), # numero de etiquetas
    id2label = {str(i): c for i, c in enumerate(labels)}, # id a etiqueta
    label2id = {c: str(i) for i, c in enumerate(labels)} # etiqueta a id
)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bia

In [15]:
from transformers import TrainingArguments

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir = "/RaymundoSGlz/distilroberta-base-mrpc-glue", # ruta donde se guardaran los checkpoints
    evaluation_strategy = "steps", # evaluamos por pasos
    num_train_epochs = 3, # numero de epocas
    push_to_hub = True, # subimos el modelo al hub
    load_best_model_at_end = True # cargamos el mejor modelo al final
)

### Entrenamiento

In [16]:
from transformers import Trainer

# Creamos el trainer
trainer = Trainer(
    model = model, # modelo
    args = training_args, # argumentos de entrenamiento
    train_dataset = prepared_ds["train"], # dataset de entrenamiento
    eval_dataset = prepared_ds["validation"], # dataset de validacion
    data_collator = data_collator, # collator
    tokenizer = tokenizer, # tokenizador
    compute_metrics = compute_metrics # metrica
)

/RaymundoSGlz/distilroberta-base-mrpc-glue is already a clone of https://huggingface.co/RaymundoSGlz/distilroberta-base-mrpc-glue. Make sure you pull the latest changes with `repo.git_pull()`.


In [17]:
# Entrenamos
train_results = trainer.train()
# Guardamos el modelo
trainer.save_model()
# Mostramos las metricas
trainer.log_metrics("train", train_results.metrics)
# Guardamos las metricas
trainer.save_metrics("train", train_results.metrics)

/root/.local/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.490900,0.544765,0.860294,0.899471
1000,0.314800,0.675343,0.843137,0.887324


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/313M [00:00<?, ?B/s]

To https://huggingface.co/RaymundoSGlz/distilroberta-base-mrpc-glue
   3f27199..44c713d  main -> main



***** train metrics *****
  epoch                    =        3.0
  total_flos               =   191920GF
  train_loss               =     0.3433
  train_runtime            = 0:03:44.27
  train_samples_per_second =     49.065
  train_steps_per_second   =       6.14


### Evaluación

In [18]:
# Evaluamos
metrics = trainer.evaluate(prepared_ds["validation"])
# Mostramos las metricas
trainer.log_metrics("eval", metrics)
# Guardamos las metricas
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.8603
  eval_f1                 =     0.8995
  eval_loss               =     0.5448
  eval_runtime            = 0:00:02.99
  eval_samples_per_second =    136.236
  eval_steps_per_second   =     17.029


### Compartimos en el Hub

In [19]:
kwargs = {
    "finetuned_from": model.config.name_or_path, # modelo base
    "tasks": "text-classification", # tareas
    "dataset": ["glue", "mrpc"], # dataset
    "tags": ["text-classification", "glue", "mrpc"] # tags
}

# Subimos el modelo al hub
trainer.push_to_hub(
    commit_message = "Entrenamiento de MRPC con DistilRoberta", # mensaje del commit
    **kwargs # argumentos
)

To https://huggingface.co/RaymundoSGlz/distilroberta-base-mrpc-glue
   44c713d..00f3cf5  main -> main

To https://huggingface.co/RaymundoSGlz/distilroberta-base-mrpc-glue
   00f3cf5..d0fb970  main -> main



'https://huggingface.co/RaymundoSGlz/distilroberta-base-mrpc-glue/commit/00f3cf529c92b269642dabba10e2b2250fb36b2f'